In [6]:
import numpy as np

ftype = np.float64

pi = ftype('3.141592653589793238462643383279502884')
sqrt2 = np.sqrt(ftype('2.0'))
sqrt3 = np.sqrt(ftype('3.0'))
f0 = ftype('0.0')
f1 = ftype('1.0')
f2 = ftype('2.0')
f3 = ftype('3.0')
f4 = ftype('4.0')
f6 = ftype('6.0')

r0 = sqrt3**-1
lambdasq = pi/f6*(r0**-2)
gamma = np.sqrt(pi / f6)
mu = np.sqrt(sqrt3 * pi) / sqrt2

delta = f1 / f4 * (r0**-4) * (
    -(mu+f2*gamma) +
    np.sqrt(mu**2 - f4*mu*gamma + f4*(gamma**2) + f4*f4*sqrt2*(gamma**2))
)

omega = f1 / f2 * (r0**-4) * (
    f3 - f2*gamma - mu - f2*(r0**4)*delta
)

In [7]:
def forward(x,y,b,ind):
    p = (
        gamma * x + (1 - gamma) * (r0 ** -2) * (x ** 3) + (r0 ** 2 - x ** 2) * x * np.sum(
            b * (x**(2*ind[0,:])) * (y**(2*ind[1,:]))
        )
    )
    e = (
        gamma * y + (1 - gamma) * (r0 ** -2) * (y ** 3) + (r0 ** 2 - y ** 2) * y * np.sum(
            b * (y**(2*ind[0,:])) * (x**(2*ind[1,:]))
        )
    )
    return np.array([p,e],dtype=ftype)

In [8]:
# Spherical projection
def sproj(A,B,C,D,E):
    return A*(B*B*C)/((C*C+D*D+E*E)**1.5)

def area(P1,P2,P3):
    """
    P1, P2, P3 - arrays of shape (2,batch) of P,E
    """
    # By Kahan: https://www.johndcook.com/blog/2020/02/27/numerical-heron/
    
    # Sort a, b, c into descending order
    if len(P1.shape) < 2:
        P1 = np.expand_dims(P1, axis=1)
        P2 = np.expand_dims(P2, axis=1)
        P3 = np.expand_dims(P3, axis=1)
        
    P1 = np.transpose(P1, axes=[1,0])
    P2 = np.transpose(P2, axes=[1,0])
    P3 = np.transpose(P3, axes=[1,0])
    P1, P2, P3 = (
        np.sqrt(np.sum((P2-P1)**2, axis=1)),
        np.sqrt(np.sum((P3-P2)**2, axis=1)),
        np.sqrt(np.sum((P1-P3)**2, axis=1))
    )
    
    P = np.stack([P1,P2,P3], axis=0)
    P = np.sort(P, axis=0)[::-1]
    
    P1 = P[0]
    P2 = P[1]
    P3 = P[2]

    p = (P1 + (P2 + P3))*(P3 - (P1 - P2))*(P3 + (P1 - P2))*(P1 + (P2 - P3))
    p = np.sqrt(p) / f4
    if p.shape[0] == 1:
        p = p.squeeze(axis=0)
    return p

# RM = 1.0
# XM = YM = R0
# XIN = YIN = XM / (6.0-1.0) for 6 steps
# DX = DY = XM * 0.1^12
def cntary(b,ind,cnt,DX,DY,XIN,YIN,XM,YM,RM,R0):
    SA = np.zeros((cnt,cnt))
    
    X = f0
    for J in range(0,cnt-1):
        Y = f0
        for I in range(0,J+1):
            # Direct transformation
            P1 = forward(X,    Y, b, ind)
            P2 = forward(X+DX, Y, b, ind)
            P3 = forward(X+DX, Y+DY, b, ind)
            P4 = forward(X,    Y+DY, b, ind)
            # Compute area of transformed square
            S1 = area(P1,P2,P3)
            S2 = area(P1,P4,P3)
            S = S1 + S2
            # Project to spherical surface
            proj = sproj(S,RM,R0,P1[0],P1[1])
            SA[I,J] = proj
            Y = Y + YIN
        X = X + XIN
    
    X = f0
    Y = YM
    
    for I in range(0,cnt):
        P1 = forward(X,    Y, b, ind)
        P2 = forward(X-DX, Y, b, ind)
        P3 = forward(X-DX, Y-DY, b, ind)
        P4 = forward(X,    Y-DY, b, ind)
        S1 = area(P1,P2,P3)
        S2 = area(P1,P4,P3)
        S = S1 + S2
        proj = sproj(S,RM,R0,P1[0],P1[1])
        SA[I,cnt-1] = proj
        X = X + XIN
    
    return SA

In [13]:
def loss(b, ind, approx, cnt):
    b[0] = f0
    b = b.reshape((approx,approx))
    
    RM = f1
    XM = r0
    YM = r0
    XIN = XM / (ftype(cnt)-f1)
    YIN = YM / (ftype(cnt)-f1)
    DX = XM * ftype('1e-12')
    DY = YM * ftype('1e-12')
    
    SA = cntary(b,ind,cnt,DX,DY,XIN,YIN,XM,YM,RM,r0)
    SAC = SA[0,0]
    SA = SA / SAC - f1
    return np.linalg.norm(SA)

In [14]:
from scipy.optimize import differential_evolution as minimize

cnt = 6
approx = 3
b0 = np.ones((approx*approx,),dtype=ftype)
ind = np.indices((approx,approx))

bounds = np.repeat(np.array([[-10.0,10.0]],dtype=ftype),approx*approx,axis=0)

iteration = 1
def callback_func(x, convergence):#, f, context
    global iteration
    print(f'[{iteration}]: {x}')
    iteration = iteration + 1

#br = minimize(loss, b0, minimizer_kwargs=dict(args=(ind,approx,cnt)), callback=callback_func)
br = minimize(loss, bounds, x0=b0, tol=0.0001, args=(ind,approx,cnt), callback=callback_func)
b = br.x

[1]: [-1.54763507  1.66324996  0.54063026 -3.62453754 -3.47517067  0.73327161
  8.69509917  1.43420083 -8.84011765]
[2]: [-1.54763507  1.66324996  0.54063026 -3.62453754 -3.47517067  0.73327161
  8.69509917  1.43420083 -8.84011765]
[3]: [-6.13978897 -0.53688988  4.07161595 -1.83131069 -1.31408958 -3.89903865
  5.12883021 -3.62149837 -6.51851783]
[4]: [-4.88301434  0.95518231 -0.1615925  -0.55966471 -5.56245327  2.00403388
  0.02254218  1.69529988 -0.35883055]
[5]: [-4.88301434  0.95518231 -0.1615925  -0.55966471 -5.56245327  2.00403388
  0.02254218  1.69529988 -0.35883055]
[6]: [-7.36611883 -0.68829906  5.95138502  0.0315141  -1.82152698 -2.5301703
 -3.8172159   1.4158779  -1.81666828]
[7]: [-7.36611883 -0.68829906  5.95138502  0.0315141  -1.82152698 -2.5301703
 -3.8172159   1.4158779  -1.81666828]
[8]: [-7.36611883 -0.68829906  5.95138502  0.0315141  -1.82152698 -2.5301703
 -3.8172159   1.4158779  -1.81666828]
[9]: [ 3.18317154 -0.13101898  4.09861483 -0.13406228 -4.08283513  3.849236

In [12]:
print(br)
print(loss(b,ind,approx,cnt))

     fun: 3.874178264754668
     jac: array([0.        , 0.00913585, 0.00865725, 0.01327045, 0.00153668,
       0.02030807, 0.00497486, 0.02525504, 0.02392233])
 message: 'Optimization terminated successfully.'
    nfev: 2050
     nit: 13
 success: True
       x: array([-5.57875028, -0.85807449,  4.91221094, -0.626749  , -1.52894425,
        3.89490986, -1.55786066,  0.66411811, -5.75417144])
3.874178264754668


In [ ]:
# Function checks
# f(-x, y) = -f( x, y)
# f( x,-y) =  f( x, y)
# f( 0, y) =  0
# f(r0, y) = r0

bl = b.reshape((3,3))
print(f'r0: {r0}')
print(f'f(-x, y) = -f( x, y): {forward(ftype(-0.1),ftype(0.1),bl,ind)} = {-forward(ftype(0.1),ftype(0.1),bl,ind)}')
print(f'f( x,-y) =  f( x, y): {forward(ftype(0.1),ftype(-0.1),bl,ind)} = {forward(ftype(0.1),ftype(0.1),bl,ind)}')
print(f'f( 0, y) =  0       : {forward(ftype(0.0),ftype(0.1),bl,ind)} = 0.0')
print(f'f(r0, y) = r0       : {forward(r0,ftype(0.1),bl,ind)}  = {r0}')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = 15 * np.random.random(100)
xdata = np.sin(zdata) + 0.1 * np.random.randn(100)
ydata = np.cos(zdata) + 0.1 * np.random.randn(100)
ax.scatter3D(xdata, ydata, zdata, c=zdata, cmap='Greens');